<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/SVC/SVC_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by SVC and TfIdfVectorizer**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from joblib import dump

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

**1- Import Data set**

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68690 entries, 0 to 68689
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    68312 non-null  object
 1   targe   68690 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


------
shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
53747,أخبارنا المغربية مروان بنصالح تداولت صفحات متع...,0
33311,أعلن الاتحاد الأوروبي لكرة القدم اليوم الخميس ...,4
16495,ينشغل حاليا الشاب خالد بالتحضير لعمل غنائي جدي...,0
7133,أخبارنا المغربية جرى إيداع هولدينغ تعود ملكيته...,2
58243,بطالة استقر معدل البطالة عند وطنيا خلال السنة ...,2
...,...,...
64381,دعا الحزب الاشتراكي الموحد المسؤولين المغاربة ...,3
15338,سخرت رواد مواقع التواصل الاجتماعى فيس بوك تويت...,4
5387,لقيت سيدة حتفها على الطريق الرابطة بين تطوان و...,1
3064,النجم الإماراتي طلب مليونا مقابل المشاركة واشت...,0


------
**2- Clean Data**

تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
53747  اخبارنا المغربيه مروان بنصالح تداولت صفحات متع...      0
33311  اعلن الاتحاد الاوروبي لكره القدم اليوم الخميس ...      4
16495  ينشغل حاليا الشاب خالد بالتحضير لعمل غنائي جدي...      0
7133   اخبارنا المغربيه جري ايداع هولدينغ تعود ملكيته...      2
58243  بطاله استقر معدل البطاله عند وطنيا خلال السنه ...      2
...                                                  ...    ...
64381  دعا الحزب الاشتراكي الموحد المسؤولين المغاربه ...      3
15338  سخرت رواد مواقع التواصل الاجتماعي فيس بوك تويت...      4
5387   لقيت سيده حتفها علي الطريق الرابطه بين تطوان و...      1
3064   نجم الاماراتي طلب مليونا مقابل المشاركه واشترط...      0
23600  اخبارنا المغربيه متابعه عمدت الاداره العامه لج...      2

[68312 rows x 2 columns]


----
 tokenizer
 
تقسيم كل نص على مستوى الكلمة

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
clean_df["text"] = clean_df["text"].apply(tokenizer.tokenize)
print(clean_df)

                                                    text  targe
53747  [اخبارنا, المغربيه, مروان, بنصالح, تداولت, صفح...      0
33311  [اعلن, الاتحاد, الاوروبي, لكره, القدم, اليوم, ...      4
16495  [ينشغل, حاليا, الشاب, خالد, بالتحضير, لعمل, غن...      0
7133   [اخبارنا, المغربيه, جري, ايداع, هولدينغ, تعود,...      2
58243  [بطاله, استقر, معدل, البطاله, عند, وطنيا, خلال...      2
...                                                  ...    ...
64381  [دعا, الحزب, الاشتراكي, الموحد, المسؤولين, الم...      3
15338  [سخرت, رواد, مواقع, التواصل, الاجتماعي, فيس, ب...      4
5387   [لقيت, سيده, حتفها, علي, الطريق, الرابطه, بين,...      1
3064   [نجم, الاماراتي, طلب, مليونا, مقابل, المشاركه,...      0
23600  [اخبارنا, المغربيه, متابعه, عمدت, الاداره, الع...      2

[68312 rows x 2 columns]


---
Delete StopWords

حذف الكلمات غير المؤثرة

In [6]:
%%time
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
print(listToStr)
#-------------
clean_df["text"] = clean_df["text"].apply(lambda x: [item for item in x if item not in stopwords_list])
((clean_df))

إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هنا

---------
accuracy calculation function

عمل تابع لحساب الدقة

In [7]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:** TfIdtVectorizer


استخراج الميزات(تحويل النص إلى أرقام)


**4- classification:** SVC

تصنيف البيانات

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [8]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 1),
    max_features =10000)

model = Pipeline([
                    ("tfidf", word_vectorizer), # convert words to numbers using tfidf
                    ("clf", SVC()) # model the text
])

from sklearn.model_selection import KFold
kf = KFold(n_splits=4,shuffle=False)
result_Pipeline=[]
# for i in range(0,len(clean_df)):
y1 = clean_df['targe']
X1 = clean_df['text']

kf.split(X1)
j =0
for train_index, test_index in kf.split(X1):
#     # Split train-test
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    model.fit(X_train.astype('str'), y_train)
    dump(model, "svm_model.joblib")
    
    y_pred=(model.predict(X_test.astype('str')))
    result_Pipeline.append(calculate_results(y_test, y_pred))

result_Pipeline

CPU times: user 3h 57min 16s, sys: 13.7 s, total: 3h 57min 30s
Wall time: 3h 57min 17s


حساب متوسط النتائج للـ5 مراحل من

 cross validation 

In [9]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_Pipeline:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_Pipeline)
f1=f1/len(result_Pipeline)
precision=precision/len(result_Pipeline)
recall=recall/len(result_Pipeline)
# (result_DecisionTreeClassifier)
results_Pipeline=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_Pipeline

[{'accuracy': 95.09310223679587,
  'f1': 0.9509164842032047,
  'precision': 0.9509299317729841,
  'recall': 0.9509310223679588}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('Pipeline results in dataset 1:')
results_Pipeline

Pipeline results in dataset 1:


[{'accuracy': 96.89533114224355,
  'f1': 0.9689985341982958,
  'precision': 0.9690937439771347,
  'recall': 0.9689533114224355}]

In [ ]:
print('Pipeline results in dataset 2:')
results_Pipeline

Pipeline results in dataset 2:


[{'accuracy': 96.70650040411039,
  'f1': 0.9670976908703979,
  'precision': 0.9671582117740604,
  'recall': 0.9670650040411037}]

In [ ]:
print('Pipeline results in dataset 3:')
results_Pipeline

Pipeline results in dataset 3:


[{'accuracy': 95.45393519602939,
  'f1': 0.9545394323284653,
  'precision': 0.9545786090267925,
  'recall': 0.9545393519602939}]

In [ ]:
print('Pipeline results in dataset 4:')
results_Pipeline

Pipeline results in dataset 4:


[{'accuracy': 95.11652418315961,
  'f1': 0.951151582880761,
  'precision': 0.9511685305885859,
  'recall': 0.9511652418315961}]

In [10]:
Final_Result=[
  {'accuracy': 96.89533114224355,
  'f1': 0.9689985341982958,
  'precision': 0.9690937439771347,
  'recall': 0.9689533114224355}
  ,
 {'accuracy': 96.70650040411039,
  'f1': 0.9670976908703979,
  'precision': 0.9671582117740604,
  'recall': 0.9670650040411037}
  ,
 {'accuracy': 95.45393519602939,
  'f1': 0.9545394323284653,
  'precision': 0.9545786090267925,
  'recall': 0.9545393519602939}
  ,
 {'accuracy': 95.11652418315961,
  'f1': 0.951151582880761,
  'precision': 0.9511685305885859,
  'recall': 0.9511652418315961}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 2.3, CPU times: user 3h 57min 16s")
print('Final_Result:')
(Final_Result)

RAM: 2.3, CPU times: user 3h 57min 16s
Final_Result:


[{'accuracy': 96.04307273138573,
  'f1': 0.96044681006948,
  'precision': 0.9604997738416433,
  'recall': 0.9604307273138573}]